In [11]:
import pdfplumber
import pandas as pd
import re

# Función para anonimizar datos sensibles
def anonimizar_descripcion(descripcion):
    # Reemplazar patrones comunes sensibles
    descripcion = re.sub(r"\d{6,}", "[OCULTO]", descripcion)  # Reemplaza números largos
    descripcion = re.sub(r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", "[EMAIL]", descripcion)  # Correos
    descripcion = re.sub(r"\b\d{10}\b", "[TELÉFONO]", descripcion)  # Teléfonos (10 dígitos)
    return descripcion

# Función para extraer datos de un estado de cuenta en PDF
def extraer_movimientos(pdf_path):
    movimientos = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                lines = text.split("\n")
                for line in lines:
                    # Expresión regular para capturar fecha, descripción y monto
                    match = re.search(r"(\d{2}-[a-zA-Z]{3})\s+[\d\*]+\s+(.+?)\s+([\d]+\.\d{2})", line)
                    if match:
                        fecha = match.group(1).strip()
                        descripcion = match.group(2).strip()
                        debito = float(match.group(3))
                        descripcion = anonimizar_descripcion(descripcion)
                        movimientos.append((fecha, descripcion, debito))

    return pd.DataFrame(movimientos, columns=["Fecha", "Descripción", "Débito"])

# Especifica el archivo PDF a procesar
ruta_pdf = "../data/pdfs/estados_cuenta/estado-cuenta1.pdf"

# Extraer movimientos del PDF especificado
df_total = extraer_movimientos(ruta_pdf)

# Filtrar para excluir "TRANSFERENCIA" y "RETIRO"
df_filtrado = df_total[~df_total["Descripción"].str.contains(r"(TRANSFERENCIA|RETIRO)", case=False, na=False)]

# Obtener los 5 mayores gastos
top_5_gastos = df_filtrado.nlargest(5, "Débito").reset_index(drop=True)

# Agrupar y sumar los gastos recurrentes
gastos_recurrentes = df_filtrado.groupby("Descripción")["Débito"].sum().reset_index()

# Obtener los 3 establecimientos con más gasto acumulado
top_3_recurrentes = gastos_recurrentes.nlargest(3, "Débito").reset_index(drop=True)

# Mostrar resultados
print("\nTop 5 Mayores Gastos:")
print(top_5_gastos)

print("\nTop 3 Gastos Recurrentes:")
print(top_3_recurrentes)



Top 5 Mayores Gastos:
    Fecha                                        Descripción  Débito
0  06-ene                26555100 CONSUMO VISA NA TEMU KS GU   44.42
1  23-dic       903244763 PAGO MEER TRF 00000000200031928399   33.47
2  16-dic     133345913 CONSUMO VISA NA H M BRANDLIVE PMZ QU   30.69
3  24-dic  39669955 CONSUMO VISA NA ETAFASHION QUICENTRO NQU   29.99
4  08-ene   901281334 CONSUMO VISA IN EA *ELECTRONIC ARTS HE   20.99

Top 3 Gastos Recurrentes:
                                      Descripción  Débito
0             26555100 CONSUMO VISA NA TEMU KS GU   44.42
1    903244763 PAGO MEER TRF 00000000200031928399   33.47
2  133345913 CONSUMO VISA NA H M BRANDLIVE PMZ QU   30.69


C:\Users\juanf\AppData\Local\Temp\ipykernel_23304\4162706362.py:32: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtrado = df_total[~df_total["Descripción"].str.contains(r"(TRANSFERENCIA|RETIRO)", case=False, na=False)]
